# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapuskasing,49.4169,-82.4331,1.05,100,100,6.69,CA,1731541667
1,1,puerto natales,-51.7236,-72.4875,7.83,64,6,2.88,CL,1731541669
2,2,ushuaia,-54.8000,-68.3000,11.81,40,75,6.17,AR,1731541510
3,3,bilibino,68.0546,166.4372,-28.83,100,87,0.26,RU,1731541671
4,4,port-aux-francais,-49.3500,70.2167,4.23,89,100,13.62,TF,1731541672


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_vacmap = city_data_df.hvplot.points("Lng", "Lat", geo = True, color = "City", alpha = 0.6, size = "Humidity", tiles = "OSM")

# Display the map
city_vacmap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filter = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Cloudiness"] < 3)
& (city_data_df["Wind Speed"] < 4.5)
                                      ]
# Drop any rows with null values
weather_filter.dropna()

# Display sample data
weather_filter

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,khanpur,28.6453,70.6567,22.25,37,0,2.72,PK,1731541728
65,65,las lomitas,-24.7096,-60.5930,23.08,31,1,2.50,AR,1731541743
106,106,tidjikja,18.5564,-11.4271,24.07,17,0,2.10,MR,1731541791
141,141,puerto maldonado,-12.6000,-69.1833,24.22,83,0,0.00,PE,1731541832
143,143,jaisalmer,26.9147,70.9181,23.87,34,0,3.10,IN,1731541834
223,223,konarka,19.9000,86.1167,22.11,74,0,4.32,IN,1731541927
226,226,al qabil,22.5710,58.6947,21.19,75,0,0.63,OM,1731541930
269,269,kidal,18.4411,1.4078,23.76,20,0,4.14,ML,1731541980
302,302,laibin,23.7000,109.2667,25.06,69,1,0.73,CN,1731542018
349,349,bhadasar,28.2833,74.3167,21.18,28,0,3.86,IN,1731542075


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filter[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
54,khanpur,PK,28.6453,70.6567,37,
65,las lomitas,AR,-24.7096,-60.5930,31,
106,tidjikja,MR,18.5564,-11.4271,17,
141,puerto maldonado,PE,-12.6000,-69.1833,83,
143,jaisalmer,IN,26.9147,70.9181,34,
223,konarka,IN,19.9000,86.1167,74,
226,al qabil,OM,22.5710,58.6947,75,
269,kidal,ML,18.4411,1.4078,20,
302,laibin,CN,23.7000,109.2667,69,
349,bhadasar,IN,28.2833,74.3167,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
khanpur - nearest hotel: Lavish Hotel
las lomitas - nearest hotel: Hotel Lomitas
tidjikja - nearest hotel: No hotel found
puerto maldonado - nearest hotel: Puerto Palmeras
jaisalmer - nearest hotel: Jeet Mahal
konarka - nearest hotel: Hotel Sun Village
al qabil - nearest hotel: Al Wesal
kidal - nearest hotel: No hotel found
laibin - nearest hotel: No hotel found
bhadasar - nearest hotel: No hotel found
san luis - nearest hotel: Hotel Dos Venados
rawson - nearest hotel: Hotel Deportivo
manta - nearest hotel: Hotel Manta Real
bardai - nearest hotel: No hotel found
saki - nearest hotel: Star International Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
54,khanpur,PK,28.6453,70.6567,37,Lavish Hotel
65,las lomitas,AR,-24.7096,-60.5930,31,Hotel Lomitas
106,tidjikja,MR,18.5564,-11.4271,17,No hotel found
141,puerto maldonado,PE,-12.6000,-69.1833,83,Puerto Palmeras
143,jaisalmer,IN,26.9147,70.9181,34,Jeet Mahal
223,konarka,IN,19.9000,86.1167,74,Hotel Sun Village
226,al qabil,OM,22.5710,58.6947,75,Al Wesal
269,kidal,ML,18.4411,1.4078,20,No hotel found
302,laibin,CN,23.7000,109.2667,69,No hotel found
349,bhadasar,IN,28.2833,74.3167,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True,color = "City", alpha = 0.8,size = "Humidity",tiles = "OSM",
hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)